DistilBERT & ANCE

In [1]:
!pip install -U beir

     |████████████████████████████████| 49 kB 2.5 MB/s 
     |████████████████████████████████| 8.4 MB 7.6 MB/s 
     |████████████████████████████████| 85 kB 4.6 MB/s 
     |████████████████████████████████| 378 kB 45.7 MB/s 
     |████████████████████████████████| 4.4 MB 34.9 MB/s 
     |████████████████████████████████| 2.8 MB 39.3 MB/s 
     |████████████████████████████████| 1.2 MB 39.8 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 3.3 MB 40.2 MB/s 
     |████████████████████████████████| 895 kB 41.6 MB/s 
     |████████████████████████████████| 636 kB 44.3 MB/s 
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp37-cp37m-linux_x86_64.whl size=267132 sha256=edc9ec094496cb943592891491989d7b6714fc15e38327b139ace644d29bbf42
  Stored in directory: /root/.cache/pip/wheels/42/96/77/0829b8b2606f90f61ba10a51277629d2b615604e122ee932f4
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any

In [2]:
import pathlib, os
import pandas as pd
from tqdm.notebook import tqdm
from beir import util, LoggingHandler
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval import models
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from beir.datasets.data_loader import GenericDataLoader

dataset = 'scidocs'
url = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip'.format(dataset)
out_dir = os.path.join(os.getcwd(), 'datasets')
data_path = util.download_and_unzip(url, out_dir)
corpus, queries, qrels = GenericDataLoader(data_path).load(split='test') # 'test', 'train', 'dev'

/usr/local/lib/python3.7/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


/content/datasets/scidocs.zip:   0%|          | 0.00/136M [00:00<?, ?iB/s]

  0%|          | 0/25657 [00:00<?, ?it/s]

In [3]:
def eval_metrics(model_name, ndcg, _map, recall, precision):
  c_map, c_ndcg, c_pre, c_rec = 'MAP@10', 'NDCG@10', 'P@10', 'Recall@10'
  eval_dict = {c_map: [_map[c_map]], c_ndcg: [ndcg[c_ndcg]], c_pre: [precision[c_pre]], c_rec: [recall[c_rec]]}
  eval_df = pd.DataFrame(data=eval_dict)
  eval_df.index = [model_name]
  return eval_df

DistilBERT

In [4]:
model_distilbert = DRES(models.SentenceBERT('msmarco-distilbert-base-v3'), batch_size=128)
retriever_distilbert = EvaluateRetrieval(model_distilbert, score_function='cos_sim')
results_distilbert = retriever_distilbert.retrieve(corpus, queries)
ndcg_d, _map_d, recall_d, precision_d = retriever_distilbert.evaluate(
    qrels, results_distilbert, retriever_distilbert.k_values)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/201 [00:00<?, ?it/s]

In [5]:
eval_metrics('DistilBERT', ndcg_d, _map_d, recall_d, precision_d)

,MAP@10,NDCG@10,P@10,Recall@10
DistilBERT,0.07561,0.13328,0.0683,0.13832


ANCE

In [6]:
model_ance = DRES(models.SentenceBERT('msmarco-roberta-base-ance-firstp'))
retriever_ance = EvaluateRetrieval(model_ance, score_function='dot')
results_ance = retriever_ance.retrieve(corpus, queries)
ndcg_a, _map_a, recall_a, precision_a = retriever_ance.evaluate(
    qrels, results_ance, retriever_ance.k_values)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/787 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.14k [00:00<?, ?B/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/201 [00:00<?, ?it/s]

In [7]:
eval_metrics('ANCE', ndcg_a, _map_a, recall_a, precision_a)

,MAP@10,NDCG@10,P@10,Recall@10
ANCE,0.06951,0.12187,0.0609,0.1236


end of fun.